# Multivariate Regression

# Show the collections that have been generated

In [1]:
from pymongo import MongoClient
from process_collection import *
#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'random_multivariate8'
client = MongoClient(mongoServer)
################################################

db = client[database]
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionNames = sorted(db.collection_names())
#collectionNames

# Formats parameters to be used for multivariate regression

In [2]:
import numpy as np
import statsmodels.api as sm
from get_cso import * 
from process_collection import * 
collectionName = collection #from cell above
ratio = 0.05
numSubcatch = 7
results = volume_reduction_and_runoff_volume_vs_lid_number(collectionName, numSubcatch, db)
csoResults = cso_reduction(collectionName, ratio, numSubcatch, results) 

### regression
y = csoResults["csoReduction"]
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    statistics = sm.OLS(y, X).fit()
    return statistics
reg_m(y,x)
print reg_m(y, x).summary()
print reg_m(y, x).rsquared


[0.0, 24223339.074383974, 24323942.3437078, 18721748.001421094, 16873402.771764398, 22963997.272643924, 16070421.31340456, 16366171.900175571, 23559477.334776998, 17908176.370896816, 16331169.79904449, 15686448.465960264, 21978242.773020267, 18443804.79250896, 21489115.507884502, 26594224.80786109, 12246844.029696822, 20303877.981420994, 22131635.32393253, 15981660.455291986, 23764043.37495649, 15074904.735180497, 24804398.190060735, 23886150.14707291, 26315197.67160082, 29023709.942808032, 19032895.742507815, 24410804.645844817, 16637463.430704236, 18209770.739891768, 27873784.211004257]
[[0, 43, 35, 48, 7, 30, 28, 11, 38, 3, 9, 6, 20, 40, 30, 1, 48, 7, 18, 20, 34, 1, 30, 16, 49, 46, 31, 13, 14, 33, 47], [0, 14, 14, 31, 38, 34, 1, 6, 26, 25, 22, 17, 35, 34, 26, 41, 5, 16, 5, 8, 41, 7, 35, 15, 41, 15, 35, 45, 38, 20, 25], [0, 45, 25, 24, 14, 27, 24, 23, 41, 39, 17, 13, 37, 6, 17, 43, 0, 37, 21, 16, 15, 42, 26, 45, 6, 29, 4, 7, 30, 43, 41], [0, 34, 40, 38, 15, 37, 5, 46, 37, 32, 15, 12,

In [3]:
### regression for total cso outflow
y = csoResults["cso"]
print y
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    statistics = sm.OLS(y, X).fit()
    return statistics
reg_m(y,x)
print reg_m(y, x).summary()
print reg_m(y, x).rsquared


[687453499.1268364, 663230160.0524524, 663129556.7831286, 668731751.1254153, 670580096.355072, 664489501.8541925, 671383077.8134319, 671087327.2266608, 663894021.7920594, 669545322.7559396, 671122329.3277919, 671767050.6608762, 665475256.3538162, 669009694.3343275, 665964383.6189519, 660859274.3189753, 675206655.0971396, 667149621.1454154, 665321863.8029039, 671471838.6715444, 663689455.7518799, 672378594.3916559, 662649100.9367757, 663567348.9797635, 661138301.4552356, 658429789.1840284, 668420603.3843286, 663042694.4809916, 670816035.6961322, 669243728.3869447, 659579714.9158322]
[[0, 43, 35, 48, 7, 30, 28, 11, 38, 3, 9, 6, 20, 40, 30, 1, 48, 7, 18, 20, 34, 1, 30, 16, 49, 46, 31, 13, 14, 33, 47], [0, 14, 14, 31, 38, 34, 1, 6, 26, 25, 22, 17, 35, 34, 26, 41, 5, 16, 5, 8, 41, 7, 35, 15, 41, 15, 35, 45, 38, 20, 25], [0, 45, 25, 24, 14, 27, 24, 23, 41, 39, 17, 13, 37, 6, 17, 43, 0, 37, 21, 16, 15, 42, 26, 45, 6, 29, 4, 7, 30, 43, 41], [0, 34, 40, 38, 15, 37, 5, 46, 37, 32, 15, 12, 42, 19

In [4]:
### regression for total volume outflow
y = csoResults["totalVol"]
print y
x = results['numLists']
print x
def reg_m(y, x):
    ones = np.ones(len(x[0]))
    X = sm.add_constant(np.column_stack((x[0], ones)))
    for ele in x[1:]:
        X = sm.add_constant(np.column_stack((ele, X)))
    statistics = sm.OLS(y, X).fit()
    return statistics
reg_m(y,x)
print reg_m(y, x).summary()
print reg_m(y, x).rsquared

[1949310793.2803576, 1909287981.0672224, 1909497650.3181446, 1918695622.1569028, 1921883768.9133675, 1911664549.0037, 1923116705.9756114, 1922469163.738426, 1910475770.1990576, 1919685564.251623, 1922697374.2062314, 1923716892.0328724, 1913083396.4372857, 1919482829.442736, 1914265274.4622254, 1905364211.1947508, 1929696952.8741238, 1915685926.197242, 1913088136.094764, 1923287555.8160474, 1910693538.6089988, 1924114659.7072465, 1908684644.2149358, 1909754897.9204257, 1906696196.326202, 1901737531.6701698, 1918490409.7997866, 1909676424.2734199, 1921871744.7255144, 1919088320.2828696, 1903503754.2551582]
[[0, 43, 35, 48, 7, 30, 28, 11, 38, 3, 9, 6, 20, 40, 30, 1, 48, 7, 18, 20, 34, 1, 30, 16, 49, 46, 31, 13, 14, 33, 47], [0, 14, 14, 31, 38, 34, 1, 6, 26, 25, 22, 17, 35, 34, 26, 41, 5, 16, 5, 8, 41, 7, 35, 15, 41, 15, 35, 45, 38, 20, 25], [0, 45, 25, 24, 14, 27, 24, 23, 41, 39, 17, 13, 37, 6, 17, 43, 0, 37, 21, 16, 15, 42, 26, 45, 6, 29, 4, 7, 30, 43, 41], [0, 34, 40, 38, 15, 37, 5, 46,